# Connected Agents

## Install Packages

In [1]:
%pip install azure-ai-projects==1.0.0b12
%pip install azure-identity
%pip install azure-ai-agents==1.1.0b3

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Import the libraries

In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole, ConnectedAgentTool

## Create the AI Project Client

NOTE: 
- If you don't have Azure CLI, you first need to install it:
   curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash
- Login to Azure
   az login



In [3]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Setup Connected Agent Tools

In [5]:
# Specify the proper agent IDs for the connected agents
search_agent = project_client.agents.get_agent("asst_XVcyiKtQiNBX3KFBK7WQ2mYB")
report_agent = project_client.agents.get_agent("asst_lU6yr86tkejQzhyuaa1LJekm")
validation_agent = project_client.agents.get_agent("asst_rmQqErBF9GuQXPU2ulpe2qzB")

search_connected_agent = ConnectedAgentTool(
        id=search_agent.id, name="search_health_plans", description="Searches for information about health plans."
    )
report_connected_agent = ConnectedAgentTool(
        id=report_agent.id, name="create_report", description="Generates reports on the health plan."
    )
validation_connected_agent = ConnectedAgentTool(
        id=validation_agent.id, name="validate_report", description="Validates the health plan report."
    )

## Create the agent

In [6]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="my-healthreport-agent",  # Name of the agent
    instructions="""
        You are an agent designed to create detailed reports about health plans. 
        The user will provide the name of a health plan and you will create a detailed report about that health plan. 
        You will also need to validate that the report meets requirements. 
        Call the appropriate agents to help write the report. 
        
        Do not write the report on your own. 
        Your role is to be an orchestrator who will call the appropriate functions provided to you. 
        Each function that you have available is an agent that can accomplish a specific task. 
        
        Here are descriptions of the plugins you have available:
            - Report Agent: An agent that writes detailed reports about health plans.
            - Search Agent: An agent that searches health plan documents.
            - Validation Agent: An agent that runs validation checks to ensure the generated report meets requirements. 
            
        Your response must contain the final approved report.
    """,  # Instructions for the agent
    tools=search_connected_agent.definitions + report_connected_agent.definitions + validation_connected_agent.definitions,  # Tools available to the agent
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_t6BbMTalK8vVKgg2Sj78tOcY


## Create thread

In [7]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_9ZFoTabIbiysCMKV5hhZK23p


## Helper function
- adds messages to the thread
- run the agent
- display the agent response

In [8]:
def run_agent(user_input):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Run the agent

In [9]:
user_input = "Create a report about the Northwind Health Plus"
run_agent(user_input)

Created message, ID: msg_H8ybWt8mwjkevFp21cWFtzAC
Run finished with status: completed
Agent Response: ### Final Approved Report on Northwind Health Plus

#### 1. Covered Services
- **Doctor Visits:** Includes both primary care physicians and specialists.
- **Preventive Care:** Immunizations, screenings, and annual wellness visits.
- **Hospital Stays:** Inpatient care and surgeries.
- **Maternity Care:** Prenatal, delivery, and postnatal services.
- **Mental Health Services:** Therapy and counseling sessions.
- **Rehabilitative Services:** Physical, occupational, and speech therapy.
- **Laboratory Tests:** Blood tests, X-rays, and other diagnostic services.
- **Prescription drugs:** Both brand name and generic medications, including over-the-counter drugs when prescribed by a doctor  .
- **Orthognathic Surgery:** Jaw augmentation or reduction  .
- **Infusion therapy services:** Available when medically necessary  .
- **Massage therapy:** Covered when medically necessary and prescribed b

## Cleanup resources

In [ ]:
# Cleanup resources

# Delete the agent after use
project_client.agents.delete_agent(agent.id)
print("Deleted agent")

Deleted agent
